<a href="https://colab.research.google.com/github/epsit03/ML-with-nvidia-rapids/blob/main/cuML-Nvidia-Rapids-conda-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [4]:
!nvidia-smi

Thu Sep  7 15:23:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup:
Set up script installs
1. Updates gcc in Colab
1. Installs Conda
1. Install RAPIDS' current stable version of its libraries, as well as some external libraries including:
  1. cuDF
  1. cuML
  1. cuGraph
  1. cuSpatial
  1. cuSignal
  1. BlazingSQL
  1. xgboost
1. Copy RAPIDS .so files into current working directory, a neccessary workaround for RAPIDS+Colab integration.


In [5]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
  Using cached pynvml-11.5.0-py3-none-any.whl (53 kB)
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS via pip!  Please stand by, should be quick...
***********************************************************************



In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/ubuntu-toolchain-r/test/ubuntu/ jammy main'
Description:
Toolchain test builds; see https://wiki.ubuntu.com/ToolChain

More info: https://launchpad.net/~ubuntu-toolchain-r/+archive/ubuntu/test
Adding repository.
Found existing deb entry in /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding deb entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Found existing deb-src entry in /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding key to /etc/apt/trusted.gpg.d/ubuntu-toolchain-r-ubuntu-test.gpg with fingerprint 60C317803A41BA51845E371A1E9377A2BA9EF27F
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  I

In [1]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [3]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

Found existing installation: cffi 1.15.0
Uninstalling cffi-1.15.0:
  Successfully uninstalled cffi-1.15.0
  Using cached cffi-1.15.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (446 kB)
Installing RAPIDS Stable 23.04
Starting the RAPIDS install on Colab.  This will take about 15 minutes.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working...
  - conda-forge/noarch::anyio-3.7.1-pyhd8ed1ab_0, conda-forge/noarch::fastapi-0.103.1-pyhd8ed1ab_0
  - conda-forge/noarch::anyio-4.0.0-pyhd8ed1ab_0, conda-forge/noarch::fastapi-0.103.0-pyhd8ed1ab_0done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=11.8
    - dask-sql
    - gcsfs
    - llvmlite
    - openssl
    - python=3.10
    - rapids=23.04


The following NEW packages will be INS

# RAPIDS is now installed on Colab.  You can copy your code into the cells below.  Enjoy!

In [4]:
import cudf, cuml, cugraph, cuspatial

In [5]:
cudf.__version__

'23.04.01'

In [7]:
import cudf
from cuml import make_regression, train_test_split
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression

In [8]:
n_samples = 2**10
n_features = 399

random_state = 23

In [9]:
## Lets generate some random regression data

%%time
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=random_state)

X = cudf.DataFrame(X)
y = cudf.DataFrame(y)[0]

X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)

CPU times: user 7.05 s, sys: 2.74 s, total: 9.79 s
Wall time: 24.5 s


In [10]:

# Copy dataset from GPU memory to host memory.
# This is done to later compare CPU and GPU results.
X_train = X_cudf.to_pandas()
X_test = X_cudf_test.to_pandas()
y_train = y_cudf.to_pandas()
y_test = y_cudf_test.to_pandas()

In [11]:
y_train

361    -85.167145
827    224.439575
449     81.067795
637   -121.376869
681    125.091248
          ...    
636     67.527672
111     65.826965
370    -45.145653
793   -192.539856
862     83.689346
Name: 0, Length: 820, dtype: float32

In [13]:
%%time
ols_sk = skLinearRegression(fit_intercept=True,
                            # normalize=True,
                            n_jobs=-1)

ols_sk.fit(X_train, y_train)

CPU times: user 75.3 ms, sys: 69.9 ms, total: 145 ms
Wall time: 147 ms


LinearRegression(n_jobs=-1)

In [14]:
%%time
predict_sk = ols_sk.predict(X_test)

CPU times: user 7.02 ms, sys: 1 ms, total: 8.02 ms
Wall time: 30.1 ms


In [15]:
%%time
r2_score_sk = r2_score(y_cudf_test, predict_sk)

CPU times: user 3.2 ms, sys: 2.08 ms, total: 5.28 ms
Wall time: 19.6 ms


In [16]:

%%time
ols_cuml = cuLinearRegression(fit_intercept=True,
                              normalize=True,
                              algorithm='eig')

ols_cuml.fit(X_cudf, y_cudf)

CPU times: user 61.1 ms, sys: 3.64 ms, total: 64.7 ms
Wall time: 125 ms


LinearRegression()

In [17]:
%%time
predict_cuml = ols_cuml.predict(X_cudf_test)

CPU times: user 104 ms, sys: 1.29 ms, total: 105 ms
Wall time: 144 ms


In [18]:

%%time
r2_score_cuml = r2_score(y_cudf_test, predict_cuml)

CPU times: user 1.19 ms, sys: 23 µs, total: 1.22 ms
Wall time: 1.23 ms


In [19]:

print("R^2 score (SKL):  %s" % r2_score_sk)
print("R^2 score (cuML): %s" % r2_score_cuml)

R^2 score (SKL):  1.0
R^2 score (cuML): 1.0


In [20]:
from sklearn.ensemble import RandomForestRegressor
from cuml.ensemble import RandomForestRegressor

# Next Steps #

For an overview of how you can access and work with your own datasets in Colab, check out [this guide](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92).

For more RAPIDS examples, check out our RAPIDS notebooks repos:
1. https://github.com/rapidsai/notebooks
2. https://github.com/rapidsai/notebooks-contrib